In [ ]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.functions_dataframe import * 
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

###### Step 1. Завантажити датафрейми ProZorro, МОН, створити список ЄДРПОУ освітніх закладів

In [ ]:
"""
Завантажити базу закупівель ProZorro (деталі у README.txt)
"""
df_prozorro = pd.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)

In [ ]:
"""
Завантажити датафрейм МОН навчальниз закладів, підготовлений у "fileter MON data.ipynb"
"""
df_mon_data_bez_upravlin_osvity = pd.read_csv('data/MON_schools_dnz.csv', sep = ';')

In [ ]:
"""
Створити список унікальних ЄДРПОУ освітніх закладів МОН
"""
mon_data_list = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

### ШКОЛИ ЗАМОВНИКИ

##### Step 2: Створити базу навчальних закладів, які самостійно купують через ProZorro

In [ ]:
"""
Відфільтрувати базу закупівель df_prozorro за колонкою IDOrganizator.
Фільтром є списко МОН з ЄДРПОУ начальних закладів mon_data_list (попередній крок)

df_tenders_by_schools - база закупівель, де навчальні заклади є замовниками
df_undefined_1 - база невизначених закупівель ProZorro
"""
df_tenders_by_schools = df_prozorro[df_prozorro['IDOrganizator'].isin(mon_data_list)]
df_undefined_1 = df_prozorro[~df_prozorro['IDOrganizator'].isin(mon_data_list)]

##### Step 3: Класифікація шкіл-замовників по типах за колонкою "Организатор"

In [ ]:
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, ['Организатор'], filter_priority, filter_name_priority)

##### Step 4: Нумерація шкіл-замовників

In [ ]:
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, ['Организатор'], reg_number)

##### Step 5: Cтворення фінальної бази ШКІЛ ЗАМОВНИКІВ

In [ ]:
"""
"Назва закладу" - колонка з фіналізованою назвою школи. Принцип визначення:
    1. Якщо колонка "Номер" не є пустою - додається до "Типу закладу"
    2. Якщо колонка "Номер" є пустою - "Назва закладу" отримує значення "Организатор"
    
* колонка "Назва" не враховується через високий ступінь повторюваності з "Организатор" і часткову точність
"""
df_tenders_by_schools = df_tenders_by_schools_numerated.copy()
df_tenders_by_schools['Номер'] = df_tenders_by_schools['Номер'].replace(r'^\s*$', np.nan, regex=True) # технічна дія: перетворити пусті строки в NaN
df_tenders_by_schools['Назва закладу'] = np.where(df_tenders_by_schools['Номер'].notnull(), 
                                                  df_tenders_by_schools['Тип закладу'] + ' ' + df_tenders_by_schools['Номер'], 
                                                  df_tenders_by_schools['Организатор'])

In [ ]:
"""
Створення колонки 'Заклад-замовник'
Видалення "технічних" колонок, створених функціями.
"""
df_tenders_by_schools['Заклад-замовник'] = True
col_names = df_prozorro.columns.tolist()
col_names += ['Заклад-замовник', 'Назва закладу']
df_tenders_by_schools = df_tenders_by_schools[col_names]

### ШКОЛИ НЕ-ЗАМОВНИКИ

##### Step 5: Пошук закупівель шкіл-не-замовників за допомогою ключових слів

In [ ]:
"""
З-поміж датафрейму невизначених закупівель (df_undefined_1) за ключовими словами у списках
filter_keywords, filter_keywords_strict відбираються закупівлі освітніх закладів у колонках: Тендер, ОписаниеТендера, Лот 

Примітка: регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки 
ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів. 
Це зменшить кількість "брудних" даних
"""
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

##### Step 6: Класифікація шкіл-не-замовників за типами

In [ ]:
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

##### Step 7: Нумерація шкіл-не-замовників

In [ ]:
"""
df_step7 - датафрейм, створений із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу"
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [ ]:
"""
Функція long_numeration створює два дафайреми:
    df_tenders_not_by_schools_numerated - тендери, де номер визначився 
    df_step7_numerated_undefined - тендери, де номер не визначився
"""
df_tenders_not_by_schools_numerated, df_step7_numerated_undefined = long_numeration(df_step7, reg_number)
df_tenders_not_by_schools_numerated['Назва закладу'] = df_tenders_not_by_schools_numerated['Тип закладу'] + ' ' + df_tenders_not_by_schools_numerated['Номер']

##### Step 8: Найменування шкіл-не-замовників

In [ ]:
"""
Завантажити датафрейм МОН власних назв навчальниз закладів, підготовлений у "fileter MON data.ipynb"
school_names - список унікальних назв 
"""
df_names_clean = pd.read_csv('data/MON_names.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()

In [ ]:
"""
Функція long_naming створює два дафайреми:
    df_tenders_not_by_schools_named - тендери, де номер визначився.
    df_tenders_not_by_schools_undefined - тендери шкіл не замовників, де не визначилась назва (і номер).
    Такі закупівлі вважаються закупівлями управінь освіти для різних закладів.
"""
df_tenders_not_by_schools_named, df_tenders_not_by_schools_undefined = long_naming(df_step7_numerated_undefined, reg_double_quotes, school_names)
df_tenders_not_by_schools_named['Назва закладу'] = df_tenders_not_by_schools_named['Тип закладу'] + ' ' + df_tenders_not_by_schools_named['Назва']

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [ ]:
"""
df_tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
Створення колонки 'Заклад-замовник'
"""
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)
df_tenders_not_by_schools['Заклад-замовник'] = False
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]

### ВСІ ШКОЛИ

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [ ]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)

In [ ]:
df_tenders.to_csv('result/df_tenders.csv', sep=';', index=False)